In [51]:
# Importamos las librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from datetime import datetime
from datetime import time
import plotly.express as px
import os
from sklearn.impute import KNNImputer

In [52]:
# Seteamos la ruta del directorio raiz}
path = "C:/Users/diego/OneDrive/Documentos/GitHub/SPP-Prediction/Floreña Ip-10/211013 - 1/Datos"
os.chdir(path)

In [53]:
def ajuste_base(base,dia):
    
    '''
    Funcion que ajusta un archivo .csv extraidos del programa WebPlotDigitizer con las siguientes tareas.
    - Elimina NaN
    - Ajusta los valores negativos a cero
    - Convierte los segundos en tiempo a partir de una fecha
    
    Devuelve un dataframe por cada parametro    
    '''
    
    # organizamos cada variable como columnas indivivuales
    base = base.drop([0],axis=0)
    base = base.astype(float)
    Block_Height = base[["Block Height","Unnamed: 1"]].rename(columns={'Block Height': 'Time', 'Unnamed: 1': 'Block Height'}).sort_values(by=['Time'])
    Hook_Load = base[["Hook Load","Unnamed: 3"]].rename(columns={'Hook Load': 'Time', 'Unnamed: 3': 'Hook Load'}).sort_values(by=['Time'])
    WOB = base[["WOB","Unnamed: 5"]].rename(columns={'WOB': 'Time', 'Unnamed: 5': 'WOB'}).sort_values(by=['Time'])
    Torque =  base[["Torque","Unnamed: 7"]].rename(columns={'Torque': 'Time', 'Unnamed: 7': 'Torque'}).sort_values(by=['Time'])
    Flow_In =  base[["Flow In","Unnamed: 9"]].rename(columns={'Flow In': 'Time', 'Unnamed: 9': 'Flow In'}).sort_values(by=['Time'])
    Pump_Pressure =  base[["Pump Pressure","Unnamed: 11"]].rename(columns={'Pump Pressure': 'Time', 'Unnamed: 11': 'Pump Pressure'}).sort_values(by=['Time'])
    ROP =  base[["ROP","Unnamed: 13"]].rename(columns={'ROP': 'Time', 'Unnamed: 13': 'ROP'}).sort_values(by=['Time'])
    RPM = base[["RPM","Unnamed: 15"]].rename(columns={'RPM': 'Time', 'Unnamed: 15': 'RPM'}).sort_values(by=['Time'])

    # Eliminamos valores nulos
    Block_Height=Block_Height.dropna()
    Hook_Load=Hook_Load.dropna()
    WOB=WOB.dropna()
    Torque=Torque.dropna()
    Flow_In=Flow_In.dropna()
    Pump_Pressure=Pump_Pressure.dropna()
    RPM=RPM.dropna()
    ROP=ROP.dropna()

    # Eliminamos valores negativos ajustandolos a ceros
    Block_Height.loc[Block_Height["Block Height"] < 0, "Block Height"] = 0
    Block_Height.loc[Block_Height["Time"] < 0, "Time"] = 0
    Hook_Load.loc[Hook_Load["Hook Load"] < 0, "Hook Load"] = 0
    Hook_Load.loc[Hook_Load["Time"] < 0, "Time"] = 0
    WOB.loc[WOB["WOB"] < 0, "WOB"] = 0
    WOB.loc[WOB["Time"] < 0, "Time"] = 0
    Torque.loc[Torque["Torque"] < 0, "Torque"] = 0
    Torque.loc[Torque["Time"] < 0, "Time"] = 0
    Flow_In.loc[Flow_In["Flow In"] < 0, "Flow In"] = 0
    Flow_In.loc[Flow_In["Time"] < 0, "Time"] = 0
    Pump_Pressure.loc[Pump_Pressure["Pump Pressure"] < 0, "Pump Pressure"] = 0
    Pump_Pressure.loc[Pump_Pressure["Time"] < 0, "Time"] = 0
    RPM.loc[RPM["RPM"] < 0, "RPM"] = 0
    RPM.loc[RPM["Time"] < 0, "Time"] = 0
    ROP.loc[ROP["ROP"] < 0, "ROP"] = 0
    ROP.loc[ROP["Time"] < 0, "Time"] = 0
    
    # Convertimos los segundos en tiempo
    Block_Height.Time=pd.to_datetime(Block_Height.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    Hook_Load.Time=pd.to_datetime(Hook_Load.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    WOB.Time=pd.to_datetime(WOB.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    Torque.Time=pd.to_datetime(Torque.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    Flow_In.Time=pd.to_datetime(Flow_In.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    Pump_Pressure.Time=pd.to_datetime(Pump_Pressure.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    RPM.Time=pd.to_datetime(RPM.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    ROP.Time=pd.to_datetime(ROP.Time,unit="m",origin=pd.Timestamp(dia)).dt.floor("S")
    
    return(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)

In [54]:
def Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP):
    
    '''
    Funcion que realiza las siguientes tareas:
    - Organiza por tiempo los datos
    - Convierte las columnas de tiempo en indice de tipo datetimeindex
    - Realiza un resampling para cada 3 segundos.
    - Organiza en un solo dataframe todas las variables con el mismo indice
    - Si despues del resampling hay valores nulos, los imputa con KNN
    
    Devuelve un dataframe merged, resampled e imputado    
    '''
    
    Block_Height.sort_values(by=['Time'])
    Hook_Load.sort_values(by=['Time'])
    WOB.sort_values(by=['Time'])
    Torque.sort_values(by=['Time'])
    Flow_In.sort_values(by=['Time'])
    Pump_Pressure.sort_values(by=['Time'])
    RPM.sort_values(by=['Time'])
    ROP.sort_values(by=['Time'])
    
    Block_Height['Time'] = pd.to_datetime(Block_Height['Time'])
    Block_Height.set_index('Time',inplace=True)
    Hook_Load['Time'] = pd.to_datetime(Hook_Load['Time'])
    Hook_Load.set_index('Time',inplace=True)
    WOB['Time'] = pd.to_datetime(WOB['Time'])
    WOB.set_index('Time',inplace=True)
    Torque['Time'] = pd.to_datetime(Torque['Time'])
    Torque.set_index('Time',inplace=True)
    Flow_In['Time'] = pd.to_datetime(Flow_In['Time'])
    Flow_In.set_index('Time',inplace=True)
    Pump_Pressure['Time'] = pd.to_datetime(Pump_Pressure['Time'])
    Pump_Pressure.set_index('Time',inplace=True)
    RPM['Time'] = pd.to_datetime(RPM['Time'])
    RPM.set_index('Time',inplace=True)
    ROP['Time'] = pd.to_datetime(ROP['Time'])
    ROP.set_index('Time',inplace=True)
    
    Block_Height=Block_Height.resample("3S").mean()
    Hook_Load=Hook_Load.resample("3S").mean()
    WOB=WOB.resample("3S").mean()
    Torque=Torque.resample("3S").mean()
    Flow_In=Flow_In.resample("3S").mean()
    Pump_Pressure=Pump_Pressure.resample("3S").mean()
    RPM=RPM.resample("3S").mean()
    ROP=ROP.resample("3S").mean()
        
    Tabla=pd.concat([Block_Height, Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP], axis=1)
    
    imputer = KNNImputer(n_neighbors=200,weights="uniform")
    df_filled = imputer.fit_transform(Tabla)
    
    Tabla_Filled=pd.DataFrame(df_filled).merge(pd.DataFrame(Tabla.index), left_index=True, right_index=True)
    Tabla_Filled.set_index('Time',inplace=True)
    Tabla_Filled.columns=Tabla.columns
    
    return(Tabla_Filled)

In [55]:
def unir_data(df_old,df_new):
    '''
    Esta funcion une el datframe de los datos previos con el dataframe de los datos nuevos
    '''
    
    Data_Export = pd.concat([df_old, df_new], axis=1).groupby(axis=1, level=0).mean()
    return Data_Export

In [56]:
def plot_incident_df(df,size):
    '''

    Funcion que toma un dataframe con indice datetime y con columnas de cada variable y realiza las siguientes tareas:
    
    - Grafica el incident

    '''

    fig1, ax1 = plt.subplots(nrows=1, ncols=3, figsize=(12,size), sharey=True)
    fig1.suptitle("Incident Log", fontsize=22)
    fig1.subplots_adjust(top=0.85,wspace=0.2)
 
    #1st track: Block Height, Hook Load & ROP
    ax01=ax1[0].twiny()
    ax01.invert_xaxis()
    ax01.invert_yaxis()
    ax01.set_xlim(0,120)
    ax01.set_ylim([np.max(df.index), np.min(df.index)])
    ax01.spines['top'].set_position(('outward',0))
    ax01.set_xlabel("Block Height [ft]")
    ax01.plot(df["Block Height"], df.index,"--", label='Block Height [ft]', color='red')
    ax01.set_xlabel('Block Height [ft]',color='red')    
    ax01.tick_params(axis='x', colors='red')
    
    ax11=ax1[0].twiny()
    ax11.invert_xaxis()
    ax11.invert_yaxis()
    ax11.set_xlim(0,900)
    ax11.plot(df["Hook Load"], df.index, label='Hook Load [Klb]', color='blue') 
    ax11.spines['top'].set_position(('outward',40))
    ax11.set_xlabel('Hook Load [Klb]',color='blue')    
    ax11.tick_params(axis='x', colors='blue')

    ax21=ax1[0].twiny() 
    ax21.invert_xaxis()
    ax21.invert_yaxis()
    ax21.invert_yaxis()
    ax21.set_xlim(0,60)
    ax21.plot(df["ROP"], df.index, label='ROP[ft/hr]', color='magenta') 
    ax21.spines['top'].set_position(('outward',80))
    ax21.set_xlabel('ROP[ft/hr]',color='magenta')    
    ax21.tick_params(axis='x', colors='magenta')
    ax21.grid(True)
    
    #2nd track: WOB, Torque, RPM
    ax02=ax1[1].twiny()
    ax02.invert_xaxis()
    ax02.invert_yaxis()
    ax02.set_xlim(0,90)
    ax02.spines['top'].set_position(('outward',0))
    ax02.set_xlabel('WOB[KLb]', color='lime')
    ax02.plot(df.WOB, df.index,"--", label='WOB[KLb]', color='lime')
    ax02.tick_params(axis='x', colors='lime')    
    ax02.grid(True)
    
    ax12=ax1[1].twiny()
    ax12.invert_xaxis()
    ax12.invert_yaxis()
    ax12.set_xlim(0,36)
    ax12.plot(df.Torque, df.index,":", label='Torque[KLb-ft]', color='blue') 
    ax12.spines['top'].set_position(('outward',40))
    ax12.set_xlabel('Torque[KLb-ft]', color='blue')    
    ax12.tick_params(axis='x', colors='blue')
    
    ax22=ax1[1].twiny()
    ax22.invert_xaxis()
    ax22.invert_yaxis()
    ax22.set_xlim(0,180)
    ax22.plot(df["RPM"], df.index, '-',label='RPM[Rev/min]', color='dimgrey') 
    ax22.spines['top'].set_position(('outward',80))
    ax22.set_xlabel('RPM[Rev/min]',color='dimgrey')
    ax22.tick_params(axis='x', colors='dimgrey')
    
    #3rd track: Flow In & Pump Pressure
    ax03=ax1[2].twiny()
    ax03.invert_xaxis()
    ax03.invert_yaxis()
    ax03.set_xlim(0,1200)
    ax03.spines['top'].set_position(('outward',0))
    ax03.set_xlabel('Flow In[GPM]')
    ax03.plot(df["Flow In"], df.index,'--', label='DT[us/ft]', color='red')
    ax03.set_xlabel('Flow In[GPM]', color='red')    
    ax03.tick_params(axis='x', colors='red')
    ax03.grid(True)
    
    ax13=ax1[2].twiny()
    ax13.invert_xaxis()
    ax13.invert_yaxis()
    ax13.set_xlim(0,6000)
    ax13.plot(df["Pump Pressure"], df.index, label='Pump Pressure[psi]', color='blue') 
    ax13.spines['top'].set_position(('outward',40))
    ax13.set_xlabel('Pump Pressure[psi]', color='blue')    
    ax13.tick_params(axis='x', colors='blue')
    
    return(fig1)

In [57]:
def plot_incident_stuck(df,size):
    
    '''
    Funcion que toma un dataframe con indice datetime y con columnas de cada variable y realiza las siguientes tareas:
    
    - Grafica el incident
    '''
    
    fig2, ax2 = plt.subplots(nrows=1, ncols=4, figsize=(12,size), sharey=True)
    fig2.suptitle("Incident Log", fontsize=20,va="top")
    fig2.subplots_adjust(top=0.7,wspace=0.2)
 
    #1st track: Block Height, Hook Load & ROP
    ax01=ax2[0].twiny()
    ax01.invert_xaxis()
    ax01.invert_yaxis()
    ax01.set_xlim(0,120)
    ax01.set_ylim([np.max(df.index), np.min(df.index)])
    ax01.spines['top'].set_position(('outward',0))
    ax01.set_xlabel("Block Height [ft]")
    ax01.plot(df["Block Height"], df.index,"--", label='Block Height [ft]', color='red')
    ax01.set_xlabel('Block Height [ft]',color='red')    
    ax01.tick_params(axis='x', colors='red')
    
    ax11=ax2[0].twiny()
    ax11.invert_xaxis()
    ax11.invert_yaxis()
    ax11.set_xlim(0,900)
    ax11.plot(df["Hook Load"], df.index, label='Hook Load [Klb]', color='blue') 
    ax11.spines['top'].set_position(('outward',40))
    ax11.set_xlabel('Hook Load [Klb]',color='blue')    
    ax11.tick_params(axis='x', colors='blue')

    ax21=ax2[0].twiny() 
    ax21.invert_xaxis()
    ax21.invert_yaxis()
    ax21.invert_yaxis()
    ax21.set_xlim(0,60)
    ax21.plot(df["ROP"], df.index, label='ROP[ft/hr]', color='magenta') 
    ax21.spines['top'].set_position(('outward',80))
    ax21.set_xlabel('ROP[ft/hr]',color='magenta')    
    ax21.tick_params(axis='x', colors='magenta')
    ax21.grid(True)
    
    #2nd track: WOB, Torque, RPM
    ax02=ax2[1].twiny()
    ax02.invert_xaxis()
    ax02.invert_yaxis()
    ax02.set_xlim(0,90)
    ax02.spines['top'].set_position(('outward',0))
    ax02.set_xlabel('WOB[KLb]', color='lime')
    ax02.plot(df.WOB, df.index,"--", label='WOB[KLb]', color='lime')
    ax02.tick_params(axis='x', colors='lime')    
    ax02.grid(True)
    
    ax12=ax2[1].twiny()
    ax12.invert_xaxis()
    ax12.invert_yaxis()
    ax12.set_xlim(0,36)
    ax12.plot(df.Torque, df.index,":", label='Torque[KLb-ft]', color='blue') 
    ax12.spines['top'].set_position(('outward',40))
    ax12.set_xlabel('Torque[KLb-ft]', color='blue')    
    ax12.tick_params(axis='x', colors='blue')
    
    ax22=ax2[1].twiny()
    ax22.invert_xaxis()
    ax22.invert_yaxis()
    ax22.set_xlim(0,180)
    ax22.plot(df["RPM"], df.index, '-',label='RPM[Rev/min]', color='dimgrey') 
    ax22.spines['top'].set_position(('outward',80))
    ax22.set_xlabel('RPM[Rev/min]',color='dimgrey')
    ax22.tick_params(axis='x', colors='dimgrey')
    
    #3rd track: Flow In & Pump Pressure
    ax03=ax2[2].twiny()
    ax03.invert_xaxis()
    ax03.invert_yaxis()
    ax03.set_xlim(0,1200)
    ax03.spines['top'].set_position(('outward',0))
    ax03.set_xlabel('Flow In[GPM]')
    ax03.plot(df["Flow In"], df.index,'--', label='DT[us/ft]', color='red')
    ax03.set_xlabel('Flow In[GPM]', color='red')    
    ax03.tick_params(axis='x', colors='red')
    ax03.grid(True)
    
    ax13=ax2[2].twiny()
    ax13.invert_xaxis()
    ax13.invert_yaxis()
    ax13.set_xlim(0,6000)
    ax13.plot(df["Pump Pressure"], df.index, label='Pump Pressure[psi]', color='blue') 
    ax13.spines['top'].set_position(('outward',40))
    ax13.set_xlabel('Pump Pressure[psi]', color='blue')    
    ax13.tick_params(axis='x', colors='blue')
    
    #4rd track: Stuck Pipe & Stuck Pipe Proba
    ax04=ax2[3].twiny()
    ax04.invert_xaxis()
    ax04.invert_yaxis()
    ax04.set_xlim(0,1)
    ax04.spines['top'].set_position(('outward',0))
    ax04.set_xlabel('Stuck Pipe')
    ax04.fill_between(df.Pega, df.index, label="Tuberia Libre o Pegada", color='blue') 
    ax04.set_xlabel('Libre [0] - Pegado [1]', color='blue')    
    ax04.tick_params(axis='x', colors='blue')
    ax04.grid(True)
    
    ax14=ax2[3].twiny()
    ax14.invert_xaxis()
    ax14.invert_yaxis()
    ax14.set_xlim(0,1)
    ax14.plot(df["Prob Pega"], df.index,label="Prediccion Prob Pega", color='red')
    ax14.spines['top'].set_position(('outward',40))
    ax14.set_xlabel('Prediccion Prob Pega', color='red')    
    ax14.tick_params(axis='x', colors='red')
    
    return(fig2)

# Cargamos los datos de todos los archivos

In [58]:
base1 = pd.read_csv("wpd_datasets-1.csv")

dia = "2013-10-21"
Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP= ajuste_base(base1,dia)
Tabla1 = Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)


base2 = pd.read_csv("wpd_datasets-2.csv")
                 
dia = "2013-10-21"
Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP= ajuste_base(base2,dia)
Tabla2=Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)
Tabla2.head()

Data_Export = unir_data(Tabla1,Tabla2)


base3 = pd.read_csv("wpd_datasets-3.csv")
                         
dia = "2013-10-21"
Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP= ajuste_base(base3,dia)
Tabla3=Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)
Tabla3.head()

Data_Export=unir_data(Data_Export,Tabla3)
'''

base4 = pd.read_csv("wpd_datasets-4.csv")
dia = "2015-01-24"
Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP= ajuste_base(base4,dia)
Tabla4=Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)
Tabla4.head()

Data_Export=unir_data(Data_Export,Tabla4)


base5 = pd.read_csv("wpd_datasets-5.csv")
dia = "2015-01-15"
Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP= ajuste_base(base5,dia)
Tabla5=Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)
Tabla5.head()

Data_Export=unir_data(Data_Export,Tabla5)


base6 = pd.read_csv("wpd_datasets_minutes-6.csv")
dia = "2017-02-01"
Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP= ajuste_base(base6,dia)
Tabla6=Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)
Tabla6.head()

Data_Export=unir_data(Data_Export,Tabla6)

base7 = pd.read_csv("wpd_datasets_minutes-7.csv")
dia = "2017-02-01"
Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP = ajuste_base(base7,dia)
Tabla7 = Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)
Tabla7.head()

Data_Export=unir_data(Data_Export,Tabla7)

base8 = pd.read_csv("wpd_datasets_minutes-8.csv")
dia = "2017-02-01"
Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP = ajuste_base(base8,dia)
Tabla8 = Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)
Tabla8.head()

Data_Export=unir_data(Data_Export,Tabla8)

'''

'\n\nbase4 = pd.read_csv("wpd_datasets-4.csv")\ndia = "2015-01-24"\nBlock_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP= ajuste_base(base4,dia)\nTabla4=Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)\nTabla4.head()\n\nData_Export=unir_data(Data_Export,Tabla4)\n\n\nbase5 = pd.read_csv("wpd_datasets-5.csv")\ndia = "2015-01-15"\nBlock_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP= ajuste_base(base5,dia)\nTabla5=Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)\nTabla5.head()\n\nData_Export=unir_data(Data_Export,Tabla5)\n\n\nbase6 = pd.read_csv("wpd_datasets_minutes-6.csv")\ndia = "2017-02-01"\nBlock_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP= ajuste_base(base6,dia)\nTabla6=Resampling(Block_Height,Hook_Load,WOB,Torque,Flow_In,Pump_Pressure,RPM,ROP)\nTabla6.head()\n\nData_Export=unir_data(Data_Export,Tabla6)\n\nbase7 = pd.read_csv("wpd_datasets_minutes-7.csv")\ndia = "2017-02-01"\nBlock_Height,Hook_

In [59]:
Data_Export

,Block Height,Flow In,Hook Load,Pump Pressure,ROP,RPM,Torque,WOB
Time,,,,,,,,
2013-10-21 00:00:00,39.766590,941.477791,248.275328,3787.347427,0.010407,119.269518,9.637404,5.745028e-07
2013-10-21 00:00:03,39.802438,933.213587,247.014228,3767.819823,0.000000,118.797589,9.618879,5.745105e-07
2013-10-21 00:00:06,39.767522,986.522234,263.037327,4016.667882,0.012347,128.067035,10.560925,2.753748e-10
2013-10-21 00:00:09,39.628725,986.982965,263.150974,4019.718053,0.012256,128.459095,10.496119,2.785431e-10
2013-10-21 00:00:12,39.458690,989.023708,263.212653,4023.693723,0.027407,128.035879,10.174019,2.810398e-10
...,...,...,...,...,...,...,...,...
2013-10-21 09:46:09,64.252484,255.377438,79.244109,174.848391,0.039006,8.800071,18.084242,9.451898e-02
2013-10-21 09:46:12,46.877130,270.020583,91.636419,560.688209,0.000000,7.224421,16.488147,9.653341e-02
2013-10-21 09:46:15,5.304163,193.841509,109.312106,1319.104485,0.038025,0.300994,0.529800,1.203645e-01


In [60]:
p1=plot_incident_df(Data_Export,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …